Notebook servant à faire le détourage des images pour créer les images de regression.

In [1]:
import cv2
from matplotlib import pyplot as plt
%matplotlib inline
import albumentations as A

import json
import yaml

from glob import glob
from re import sub
from random import randrange
import os
from PIL import Image
import math
from tqdm import tqdm
import shutil
import numpy as np
import supervision as sv

In [2]:
src_dir = "source" # Dossier source avec sous dossiers "images" et "labels" et fichier notes.json

cut_out_res_path = "cut_out_images" # Dossier final avec les images détourées

In [3]:
# création des dossier et sous dossiers
splits = []
for split in ['test', 'val', 'train']:
    if split in os.listdir(src_dir):
        splits .append(split)

try:
    os.mkdir(cut_out_res_path)
except OSError as error:  
    print(error)
for split in splits:
    try:
        os.mkdir(os.path.join(cut_out_res_path, split))
    except OSError as error:  
        print(error)


[WinError 183] Impossible de créer un fichier déjà existant: 'cut_out_images'
[WinError 183] Impossible de créer un fichier déjà existant: 'cut_out_images\\val'
[WinError 183] Impossible de créer un fichier déjà existant: 'cut_out_images\\train'


In [6]:
def cut_out(mask, img, bbox):
    """Détoure le mask de l'image et comble la boîte de pixels noirs

    Args:
        mask (np.ndarray): mask composé de 1 et 0 définissant les pixels à détourer
        img (np.ndarray): image de référence
        bbox (list): liste de 4 coordonnées définissant la boite encadrante

    Returns:
        np.array: image de la rangée détourée
    """
    x_min, y_min, x_max, y_max = bbox
    color = np.array([0,0,0], dtype='uint8')
    cut_out_img = np.where(mask[...,None], img, color) # met les pixels correpsondans au 0 du mask en noir
    cut_out_img = cut_out_img[y_min:y_max, x_min:x_max] # coupe l'image au dimensions de la boite encadrante
    return cut_out_img

In [7]:
for split in splits:
    # dossier cibles
    img_src_path = os.path.join(src_dir, split, 'images')
    lab_src_path = os.path.join(src_dir, split, 'labels')
    img_cut_out_res_path = os.path.join(cut_out_res_path, split)
    images = glob('*.jpg', dir_fd=img_src_path)
    
    for img_name in tqdm(images, desc = 'images processed: ' + split):
        image = cv2.imread(os.path.join(img_src_path, img_name)) # lire l'image
        h, w, _ = image.shape
        category_ids = []
        label_name = sub("jpg$", "txt", img_name)
        with open(os.path.join(lab_src_path, label_name), "r") as label_file:
            lines = label_file.readlines()
            polygons = np.empty(len(lines), dtype = object) # Array des polygones des rangées de la figure
            for line_id in range(len(lines)): # Pour chaque annotation (rangée)
                split_line = lines[line_id].split(' ')
                category_ids.append(int(split_line[0]))
                split_line = list(map(float, split_line))
                polygon = np.empty(shape=(len(split_line)//2,2), dtype=np.int32) # Array des sommet du polygon de la rangée
                for i in range(1, len(split_line), 2):
                    polygon[i//2] = [split_line[i]*w, split_line[i+1]*h]
                polygons[line_id] = polygon
        masks = [ sv.polygon_to_mask(p,(w,h)) for p in polygons ] # Conversion des polygons en masks
        masks = np.array(masks)
        bboxes = [ sv.polygon_to_xyxy(p) for p in polygons] # Conversion des polygons pour avoir la boite encadrante la plus proche

        # découpage et sauvegarde
        for k in range(len(masks)):
            cut_out_img = cut_out(masks[k], image, bboxes[k])
            cv2.imwrite(os.path.join(img_cut_out_res_path, str(k) + '_' + img_name), cut_out_img)

images processed: val:   0%|          | 0/18 [00:00<?, ?it/s]

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


images processed: val:   6%|▌         | 1/18 [00:00<00:07,  2.28it/s]

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


images processed: val:  11%|█         | 2/18 [00:00<00:04,  3.46it/s]

...

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


images processed: train:  97%|█████████▋| 31/32 [00:07<00:00,  4.19it/s]

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


images processed: train: 100%|██████████| 32/32 [00:07<00:00,  4.15it/s]

<class 'numpy.ndarray'>


# Inspection / Analyse

In [6]:
print("Dimension moyenne des images de classification:")

for split in ["train", "val", "test"]:
    print("------------------------------------------\n" + split + ':')
    mean_w = 0
    mean_h = 0
    max_w, min_w, max_h, min_h = 0, math.inf , 0, math.inf
    nb_img = 0

    split_path = os.path.join(cut_out_res_path, split)
   
    fnames = glob('*.jpg', dir_fd=split_path)
    nb_img += len(fnames)
    for fname in tqdm(fnames, desc = 'image in '+split):
        with Image.open(os.path.join(split_path, fname)) as img:
            w, h = img.size
            mean_w += w
            mean_h += h
            if w > max_w:
                max_w = w
            if w < min_w:
                min_w = w
            if h > max_h:
                max_h = h
            if h < min_h:
                min_h = h
    if nb_img != 0:            
        mean_w /= nb_img
        mean_h /= nb_img

        print("\nmean width = ", mean_w)
        print("mean heigh = ", mean_h)
        print("max width: ", max_w)
        print("min width: ", min_w)
        print("max heigh: ", max_h)
        print("min heigh: ", min_h)
        print("number of images: ", nb_img)
    else:
        print("No image in " + split + " split!")

Dimension moyenne des images de classification:
------------------------------------------
train:


image in train: 100%|██████████| 128/128 [00:01<00:00, 109.77it/s]



mean width =  169.515625
mean heigh =  303.8046875
max width:  292
min width:  95
max heigh:  520
min heigh:  104
number of images:  128
------------------------------------------
val:


image in val: 100%|██████████| 77/77 [00:00<00:00, 106.91it/s]



mean width =  151.45454545454547
mean heigh =  291.53246753246754
max width:  291
min width:  70
max heigh:  515
min heigh:  87
number of images:  77
------------------------------------------
test:


image in test: 0it [00:00, ?it/s]

No image in test split!
